Python Libraries


In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

Custom Functions


In [2]:
from useful_functions.dd_dictionary import create_dd_dictionary
from useful_functions.pd_dictionary import create_pd_dictionary
from useful_functions.takeover_dataframe import create_takeover_timestamps
from useful_functions.check_for_missing_data import check_for_missing_data

# Importing Data


---


In [3]:
driving_data_folder = "../AdVitam/Exp2/Raw/Driving"
physio_data_folder = "../AdVitam/Exp2/Raw/Physio/Txt"

### Participants to Exclude

| Participant | Reason |
| --- | --- |
| NST77 | Driving file contains obstacles = "TriggeredObs2TriggeredObs3" and "TriggeredObs3TriggeredObs4" |
| NST91, ST84, ST60 | Does not contain a physio file |

In [4]:
check_for_missing_data(driving_data_folder, physio_data_folder)

['NST91.txt', 'ST84.txt', 'ST60.txt']

In [5]:
participants_to_exclude = ["NST77", 'NST91', 'ST84', 'ST60']

### Driving Data

Metadata:

- Time = Time elapsed since the software was launched (in seconds)
- EngineSpeed = Engine speed (in rpm)
- GearPosActual = Current gear
- GearPosTarget = Next planned gear
- AcceleratorPedalPos = Position of gas pedal. Should not be used (see below)
- DeceleratorPedalPos = Position of brake pedal. Should not be used (see below)
- SteeringWheelAngle = Steering wheel angle (in degrees)
- VehicleSpeed = Vehicle speed (in mph)
- Position X = Vehicle position along the x-axis in the simulated driving environment
- Position Y = Vehicle position along the y-axis in the simulated driving environment
- Position Z = Vehicle position along the z-axis in the simulated driving environment
- Autonomous Mode (T/F) = Autonomous pilot status. True = autonomous pilot activated, False = autonomous pilot deactivated (driver in control of the car)
- Obstacles: Events that occurred during the driving simulation.
  - TriggeredObsX = Time at which each takeover request was triggered by the experimenter..
  - Obs1 = deer, Obs2 = traffic cone, Obs3 = frog, Obs4 = traffic cone, Obs5 = false alarm (x2).
  - Detected = Time at which the driver pressed the steering wheel button to notify he/she understood the situation. The driver is in control of the car when the value of the column "Autonomous Mode (T/F)" is False.

Note: Due to recording problem, the "AcceleratorPedalPos" and "DeceleratorPedalPos" columns do not correspond to the gas and brake pedal position.


In [6]:
driving_data_dictionary = create_dd_dictionary(
    driving_data_folder, participants_to_exclude
)
len(driving_data_dictionary)

87

**Processing the driving data**

Steps Taken
1. Fit a label encoder to the `Obstacles` column
2. Transform the `Obstacles` column for all driver data
3. Resample driver data to 10ms

In [7]:
# Fitting a Label Encoder to the Obstacles
driver_data =driving_data_dictionary['NST01']

# label encoding
enc = LabelEncoder()
enc.fit(driver_data["Obstacles"])

for driver in driving_data_dictionary.keys():
    driver_data = driving_data_dictionary[driver]
    # label encoding
    driver_data["Obstacles"] = enc.transform(driver_data["Obstacles"])

    # resampling
    driver_data["Time"] = pd.to_timedelta(driver_data["Time"], unit="s")
    driver_data = driver_data.drop_duplicates(subset="Time")
    driver_data = driver_data.set_index("Time")
    driver_data = driver_data.resample("10ms").ffill()
    driver_data = driver_data.reset_index()

    # replacing the dictionary value
    driving_data_dictionary[driver] = driver_data

### Physiological Signals

Metadata:

- min = Time elaspsed
- ECG = Electrocardiogram (1000Hz)
- EDA = Electrodermal Activity (1000Hz)
- RESP = Resperatory (1000Hz)

### Physiological Signal Markers

contains the timestamps for each period of the experiment.

- Training1 = Baseline phase
- Training2 = Practice phase in the driving simulator
- Driving = Main driving session in conditionally automated driving.

Be careful, the timestamps are here in seconds while they are in minutes in the raw data.


In [8]:
phsyiological_data_dictionary = create_pd_dictionary(
    physio_data_folder, participants_to_exclude
)
len(phsyiological_data_dictionary) / 2

87.0

### Driver Demographic Data


In [9]:
driver_demographic_data = pd.read_csv(
    "../AdVitam/Exp2/Preprocessed/Questionnaires/Exp2_Database.csv"
)
driver_demographic_data.head()

,code,date,time,condition,sex,age,mothertongue,education,driving_license,km_year,...,sart_6_fa,sart_7_fa,sart_8_fa,sart_10_fa,demand_fa,supply_fa,understanding_fa,sart_global_fa,nb_times_remind_counting,notes
0,NST1,51218,90503,1,1,19,1,1,2017,200,...,6,7,5,7,8,27,16,35,NaN,I accidentally triggered the F9 obstacle twice...
1,ST2,51218,100823,2,1,19,1,1,2017,5000,...,1,2,1,7,3,6,12,15,1.0,NaN
2,NST3,51218,161024,1,1,19,1,1,2017,1000,...,3,6,1,2,3,17,7,21,NaN,"Timestamp driving start : Driving End, the 1st..."
3,ST4,61218,144813,2,1,21,3,2,2016,1500,...,2,4,2,2,6,12,8,14,1.0,"In the testing phase 1, we have to remove the ..."
4,NST5,61218,162147,1,1,22,1,1,2017,1500,...,4,2,4,1,3,18,9,24,NaN,NaN


---


# Defining Takeover Quality Quantitatively


- Takeover Time (TOT)
- Sudden Vehicle Deviation
- Response Budget


### Takeover Time


In [10]:
takeover_timestamps = create_takeover_timestamps(driving_data_dictionary, enc)
takeover_timestamps

,TriggeredObs1,TakeoverObs1,ReleaseObs1,TOTObs1,TriggeredObs2,TakeoverObs2,ReleaseObs2,TOTObs2,TriggeredObs3,TakeoverObs3,ReleaseObs3,TOTObs3,TriggeredObs4,TakeoverObs4,ReleaseObs4,TOTObs4,TriggeredObs5,TakeoverObs5,ReleaseObs5,TOTObs5
NST01,0 days 00:05:11.974200,0 days 00:05:18.804200,0 days 00:05:28.764200,0 days 00:00:06.830000,0 days 00:09:11.494200,0 days 00:09:13.964200,0 days 00:09:23.654200,0 days 00:00:02.470000,0 days 00:10:50.094200,0 days 00:10:54.174200,0 days 00:10:54.554200,0 days 00:00:04.080000,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
ST02,0 days 00:08:03.979300,0 days 00:08:08.999300,0 days 00:08:17.339300,0 days 00:00:05.020000,0 days 00:06:03.149300,0 days 00:06:06.569300,0 days 00:06:09.769300,0 days 00:00:03.420000,0 days 00:14:38.599300,0 days 00:14:43.159300,0 days 00:14:44.779300,0 days 00:00:04.560000,0 days 00:17:24.939300,0 days 00:17:29.289300,0 days 00:17:33.199300,0 days 00:00:04.350000,NaT,NaT,NaT,NaT
NST03,0 days 00:16:04.013200,0 days 00:16:08.633200,0 days 00:16:41.013200,0 days 00:00:04.620000,0 days 00:12:48.623200,0 days 00:12:51.843200,0 days 00:13:24.443200,0 days 00:00:03.220000,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
ST04,0 days 00:19:23.934300,0 days 00:19:36.624300,0 days 00:19:54.174300,0 days 00:00:12.690000,0 days 00:13:29.504300,0 days 00:13:32.174300,0 days 00:13:39.614300,0 days 00:00:02.670000,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
NST05,0 days 00:10:02.164780,0 days 00:10:04.474780,0 days 00:10:06.294780,0 days 00:00:02.310000,0 days 00:15:16.364780,0 days 00:15:31.474780,0 days 00:15:36.064780,0 days 00:00:15.110000,0 days 00:17:52.614780,0 days 00:17:55.954780,0 days 00:18:00.284780,0 days 00:00:03.340000,0 days 00:07:21.274780,0 days 00:07:24.604780,0 days 00:07:27.654780,0 days 00:00:03.330000,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ST86,0 days 00:07:20.262000,0 days 00:07:23.202000,0 days 00:07:40.682000,0 days 00:00:02.940000,0 days 00:05:16.102000,0 days 00:05:23.742000,0 days 00:05:27.472000,0 days 00:00:07.640000,0 days 00:16:00.142000,0 days 00:16:04.732000,0 days 00:16:07.492000,0 days 00:00:04.590000,0 days 00:19:12.122000,0 days 00:19:14.572000,0 days 00:19:20.642000,0 days 00:00:02.450000,NaT,NaT,NaT,NaT
NST87,0 days 00:12:09.407400,0 days 00:12:13.327400,0 days 00:12:19.367400,0 days 00:00:03.920000,0 days 00:10:02.517400,0 days 00:10:06.027400,0 days 00:10:11.847400,0 days 00:00:03.510000,0 days 00:04:03.947400,0 days 00:04:07.327400,0 days 00:04:19.727400,0 days 00:00:03.380000,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
ST88,0 days 00:19:38.248130,0 days 00:19:43.248130,0 days 00:19:48.678130,0 days 00:00:05,0 days 00:13:52.038130,0 days 00:13:56.968130,0 days 00:14:01.768130,0 days 00:00:04.930000,0 days 00:07:07.728130,0 days 00:07:12.438130,0 days 00:07:15.818130,0 days 00:00:04.710000,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
NST89,0 days 00:10:29.380100,0 days 00:10:31.340100,0 days 00:10:42.150100,0 days 00:00:01.960000,0 days 00:17:07.450100,0 days 00:17:09.720100,0 days 00:17:20.770100,0 days 00:00:02.270000,0 days 00:20:01.860100,0 days 00:20:05.380100,0 days 00:20:16.140100,0 days 00:00:03.520000,0 days 00:07:02.120100,0 days 00:07:05.210100,0 days 00:07:11.200100,0 days 00:00:03.090000,NaT,NaT,NaT,NaT


# !! Here!!

Why are the obstacle trigger times different in the processed data?

Could that have affected my results?

Example:

timestamps_obstacles.csv: Time elapsed (in seconds) between the start of the main driving session and the appearance of the obstacles (TrigObsX), the time when the driver pressed the button to report having understood the situation (DetObsX), and the time when the driver actually took over control (RepObsX). X corresponds to one of obstacle or the false alarm.

In [11]:
obstacle_timestamps = pd.read_csv(
    "../AdVitam/Exp2/Preprocessed/Physio and Driving/timestamps_obstacles.csv"
)
obstacle_timestamps.head()

,subject_id,label_st,TrigObsDeer,DetObsDeer,RepObsDeer,TrigObsCone,DetObsCone,RepObsCone,TrigObsFrog,DetObsFrog,RepObsFrog,TrigObsCan,DetObsCan,RepObsCan,TrigObsFA1,DetObsFA1,RepObsFA1,TrigObsFA2,DetObsFA2,RepObsFA2
0,NST1,0,176.7051,179.0932,183.5238,416.2140,418.5109,418.6902,514.8157,518.8456,NaN,786.6408,792.8591,NaN,983.6240,NaN,NaN,1082.2450,1086.3630,NaN
1,ST2,1,230.7565,234.5881,235.7780,109.9334,112.2556,113.3516,625.3827,628.3633,629.9416,791.7203,793.7173,796.0673,357.7144,360.2164,NaN,468.5158,470.4113,NaN
2,NST3,0,815.2040,818.4693,819.8244,619.8088,621.8167,623.0322,259.4712,263.7890,NaN,1027.4009,1030.7859,NaN,378.3409,NaN,NaN,1115.1749,1122.8189,NaN
3,ST4,1,1040.3619,1042.3819,1053.0479,685.9281,NaN,NaN,287.3724,289.5112,NaN,119.9266,120.5600,NaN,410.9882,462.7615,NaN,886.9369,889.8139,NaN
4,NST5,0,428.9613,430.7424,431.2726,743.1664,744.4617,758.2736,899.4186,900.7246,902.7536,268.0716,269.9875,271.4027,143.6314,144.7424,NaN,629.1736,630.4212,NaN


In [12]:
takeover_timestamps.head()

,TriggeredObs1,TakeoverObs1,ReleaseObs1,TOTObs1,TriggeredObs2,TakeoverObs2,ReleaseObs2,TOTObs2,TriggeredObs3,TakeoverObs3,ReleaseObs3,TOTObs3,TriggeredObs4,TakeoverObs4,ReleaseObs4,TOTObs4,TriggeredObs5,TakeoverObs5,ReleaseObs5,TOTObs5
NST01,0 days 00:05:11.974200,0 days 00:05:18.804200,0 days 00:05:28.764200,0 days 00:00:06.830000,0 days 00:09:11.494200,0 days 00:09:13.964200,0 days 00:09:23.654200,0 days 00:00:02.470000,0 days 00:10:50.094200,0 days 00:10:54.174200,0 days 00:10:54.554200,0 days 00:00:04.080000,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
ST02,0 days 00:08:03.979300,0 days 00:08:08.999300,0 days 00:08:17.339300,0 days 00:00:05.020000,0 days 00:06:03.149300,0 days 00:06:06.569300,0 days 00:06:09.769300,0 days 00:00:03.420000,0 days 00:14:38.599300,0 days 00:14:43.159300,0 days 00:14:44.779300,0 days 00:00:04.560000,0 days 00:17:24.939300,0 days 00:17:29.289300,0 days 00:17:33.199300,0 days 00:00:04.350000,NaT,NaT,NaT,NaT
NST03,0 days 00:16:04.013200,0 days 00:16:08.633200,0 days 00:16:41.013200,0 days 00:00:04.620000,0 days 00:12:48.623200,0 days 00:12:51.843200,0 days 00:13:24.443200,0 days 00:00:03.220000,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
ST04,0 days 00:19:23.934300,0 days 00:19:36.624300,0 days 00:19:54.174300,0 days 00:00:12.690000,0 days 00:13:29.504300,0 days 00:13:32.174300,0 days 00:13:39.614300,0 days 00:00:02.670000,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
NST05,0 days 00:10:02.164780,0 days 00:10:04.474780,0 days 00:10:06.294780,0 days 00:00:02.310000,0 days 00:15:16.364780,0 days 00:15:31.474780,0 days 00:15:36.064780,0 days 00:00:15.110000,0 days 00:17:52.614780,0 days 00:17:55.954780,0 days 00:18:00.284780,0 days 00:00:03.340000,0 days 00:07:21.274780,0 days 00:07:24.604780,0 days 00:07:27.654780,0 days 00:00:03.330000,NaT,NaT,NaT,NaT
